1. Importações

In [ ]:
import pandas as pd
from google.colab import files

2. Entrada de dados

In [ ]:
#entrada de dados
situacao_problema = pd.read_csv('/content/sample_data/situacao_problema.csv', decimal=',')
coordenadas = pd.read_csv('/content/sample_data/coordenadas.csv', decimal=',')
elementos_df = pd.read_csv('/content/sample_data/elementos_df.csv', decimal=',')
temperatura = pd.read_csv('/content/sample_data/temperatura.csv', decimal=',')
carregamentos = pd.read_csv('/content/sample_data/carregamentos.csv', decimal=',')
apoios = pd.read_csv('/content/sample_data/apoios.csv', decimal=',')

#ajuste de dados
coordenadas['x'] = coordenadas['x'].astype(float)
coordenadas['y'] = coordenadas['y'].astype(float)
elementos_df['E'] = elementos_df['E'].astype(float)
temperatura['delta'] = temperatura['delta'].astype(float)
carregamentos['Fx'] = carregamentos['Fx'].astype(float)
carregamentos['Fy'] = carregamentos['Fy'].astype(float)

situacao_problema.info()
coordenadas.info()
elementos_df.info()
temperatura.info()
carregamentos.info()
apoios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   nos                   1 non-null      int64
 1   elementos             1 non-null      int64
 2   n_elementos_var_temp  1 non-null      int64
 3   n_nos_carregados      1 non-null      int64
 4   n_apoios              1 non-null      int64
dtypes: int64(5)
memory usage: 168.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   no      4 non-null      int64  
 1   x       4 non-null      float64
 2   y       4 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 224.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  ----

3. Entrada de dados em vetores e variáveis

In [ ]:
#entrada de dados em variáveis
nos = situacao_problema.loc[situacao_problema.index == 0, 'nos'].values[0]
elementos_count = situacao_problema.loc[situacao_problema.index == 0, 'elementos'].values[0]
n_elem_var_temp = situacao_problema.loc[situacao_problema.index == 0, 'n_elementos_var_temp'].values[0]
n_nos_carregados = situacao_problema.loc[situacao_problema.index == 0, 'n_nos_carregados'].values[0]
n_apoios = situacao_problema.loc[situacao_problema.index == 0, 'n_apoios'].values[0]

#entrada de dados em vetores
no = coordenadas['no'].tolist()
x = coordenadas['x'].tolist()
y = coordenadas['y'].tolist()
elemento = elementos_df['elemento'].tolist()
no_i = elementos_df['no_i'].tolist()
no_f = elementos_df['no_f'].tolist()
E = elementos_df['E'].tolist()
A = elementos_df['A'].tolist()
beta = elementos_df['beta'].tolist()
delta = temperatura['delta'].tolist()
nos_carregados = carregamentos['no'].tolist()
Fx = carregamentos['Fx'].tolist()
Fy = carregamentos['Fy'].tolist()
nos_apoiados = apoios['no'].tolist()
vx = apoios['vx'].tolist()
vy = apoios['vy'].tolist()

4. Incidência dos elementos

In [ ]:
e1, e2, e3, e4, e = [], [], [], [], []

for i in range(elementos_count):
  e1.append(2*no_i[i]-1)
  e2.append(2*no_i[i])
  e3.append(2*no_f[i]-1)
  e4.append(2*no_f[i])
for i in range(elementos_count):
  e.append([e1[i], e2[i], e3[i], e4[i]])
print(e)

[[1, 2, 7, 8], [3, 4, 7, 8], [5, 6, 7, 8]]


5. Comprimento das barras

In [ ]:
L = []
for i in range (elementos_count):
  L.append(((x[no_f[i]-1]-x[no_i[i]-1])**2 + (y[no_f[i]-1]-y[no_i[i]-1])**2)**(1/2))
print(L)

[500.0, 375.0, 300.0]


6. Seno dos elementos

In [ ]:
sen = []
for i in range (elementos_count):
  sen.append((y[no_f[i]-1]-y[no_i[i]-1])/L[i])
print(sen)

[0.6, 0.8, 1.0]


7. Cosseno dos elementos

In [ ]:
cos = []
for i in range (elementos_count):
  cos.append((x[no_f[i]-1]-x[no_i[i]-1])/L[i])
print(cos)

[0.8, 0.6, 0.0]


8. Matrizes de rigidez local dos elementos

In [ ]:
import numpy as np
m = []
matrizes_rigidez_local = []

for i in range (elementos_count):
  m = (E[i]*A[i]/L[i]) * np.array([[cos[i]**2, sen[i]*cos[i], -cos[i]**2, -sen[i]*cos[i]],
                 [sen[i]*cos[i], sen[i]**2, -sen[i]*cos[i], -sen[i]**2],
                 [-cos[i]**2, -sen[i]*cos[i], cos[i]**2, sen[i]*cos[i]],
                 [-sen[i]*cos[i], -sen[i]**2, sen[i]*cos[i], sen[i]**2]])
  matrizes_rigidez_local.append(m)
for i, matriz in enumerate(matrizes_rigidez_local):
    print(f"Matriz {i + 1}:\n{matriz}\n")

Matriz 1:
[[ 480.  360. -480. -360.]
 [ 360.  270. -360. -270.]
 [-480. -360.  480.  360.]
 [-360. -270.  360.  270.]]

Matriz 2:
[[ 360.  480. -360. -480.]
 [ 480.  640. -480. -640.]
 [-360. -480.  360.  480.]
 [-480. -640.  480.  640.]]

Matriz 3:
[[    0.     0.    -0.    -0.]
 [    0.  1250.    -0. -1250.]
 [   -0.    -0.     0.     0.]
 [   -0. -1250.     0.  1250.]]



9. Matriz de rigidez global zerada

In [ ]:
K = np.zeros((2*nos, 2*nos))
print(K)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


10. Matriz de rigidez global

In [ ]:
for m in range(elementos_count):
  n = [2*no_i[m]-2, 2*no_i[m]-1, 2*no_f[m]-2, 2*no_f[m]-1]
  for i in range (4):
    for j in range (4):
      K[n[i]][n[j]] += matrizes_rigidez_local[m][i][j]
print(K)

[[  480.   360.     0.     0.     0.     0.  -480.  -360.]
 [  360.   270.     0.     0.     0.     0.  -360.  -270.]
 [    0.     0.   360.   480.     0.     0.  -360.  -480.]
 [    0.     0.   480.   640.     0.     0.  -480.  -640.]
 [    0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.  1250.     0. -1250.]
 [ -480.  -360.  -360.  -480.     0.     0.   840.   840.]
 [ -360.  -270.  -480.  -640.     0. -1250.   840.  2160.]]


11. Vetor do carregamento nodal térmico dos elementos

In [ ]:
#temperatura
a = []
ft = []
for i in range (n_elem_var_temp):
  a = (E[i]*A[i]*beta[i]*delta[i] * np.array([-cos[i],-sen[i],cos[i],sen[i]])).tolist()
  ft.append(a)
for i, vetor_carregamento_termico in enumerate(ft):
    print(f"ft {i + 1}:\n{vetor_carregamento_termico}\n")

ft 1:
[-240.00000000000006, -180.00000000000003, 240.00000000000006, 180.00000000000003]

ft 2:
[-180.00000000000003, -240.00000000000006, 180.00000000000003, 240.00000000000006]

ft 3:
[-0.0, -300.00000000000006, 0.0, 300.00000000000006]



In [ ]:
Ft = np.zeros((2*nos))
print(Ft)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
for i in range(n_elem_var_temp):
  n = [2*no_i[i]-2, 2*no_i[i]-1, 2*no_f[i]-2, 2*no_f[i]-1]
  for j in range (4):
    Ft[n[j]] += ft[i][j]
print(Ft)

[-240. -180. -180. -240.    0. -300.  420.  720.]


In [ ]:
#carregamento
b = []
fc = []
for i in range (n_nos_carregados):
  b = (Fx[i],Fy[i])
  fc.append(b)
for i, vetor_carregamento in enumerate(fc):
    print(f"fc {i + 1}:\n{vetor_carregamento}\n")

fc 1:
(42.0, 72.0)



In [ ]:
Fc = np.zeros((2*nos))
print(Fc)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
for i in range(n_nos_carregados):
  n = [2*nos_carregados[i]-2, 2*nos_carregados[i]-1]
  for j in range (2):
   Fc[n[j]] += fc[i][j]
print(Fc)

[ 0.  0.  0.  0.  0.  0. 42. 72.]


In [ ]:
F = np.zeros((2*nos))
print(F)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
#vetor de carregamento total nodal
for i in range (2*nos):
  F[i] = Fc[i]+Ft[i]
print(F)

[-240. -180. -180. -240.    0. -300.  462.  792.]


12. Sistema linear

In [ ]:
condicoes_contorno = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

linhas_remover = sorted(condicoes_contorno.keys())
K_reduzida = np.delete(K, linhas_remover, axis=0)
K_reduzida = np.delete(K_reduzida, linhas_remover, axis=1)
F_reduzido = np.delete(F, linhas_remover)

v_reduzido = np.linalg.solve(K_reduzida, F_reduzido)

v = np.zeros(len(F))
indice_reduzido = 0
for i in range(len(v)):
    if i in condicoes_contorno:
        v[i] = condicoes_contorno[i]
    else:
        v[i] = v_reduzido[indice_reduzido]
        indice_reduzido += 1

for i, deslocamento in enumerate(v, start=1):
    print(f"v{i} = {deslocamento:.4f}")

v1 = 0.0000
v2 = 0.0000
v3 = 0.0000
v4 = 0.0000
v5 = 0.0000
v6 = 0.0000
v7 = 0.3000
v8 = 0.2500


13. Deslocamentos nas extremidades

In [ ]:
deslocamentos_extremidade = {}

for i, indices in enumerate(e):
    deslocamentos_extremidade[i] = [v[index - 1] for index in indices]

for i, valores in deslocamentos_extremidade.items():
    print(f"e{i+1}: {valores}")

e1: [0.0, 0.0, 0.3, 0.2500000000000001]
e2: [0.0, 0.0, 0.3, 0.2500000000000001]
e3: [0.0, 0.0, 0.3, 0.2500000000000001]


14. variação do comprimento dos elementos

In [ ]:
B = []
Be = []
for i in range (elementos_count):
  B = np.array([[cos[i], sen[i], 0, 0],
                 [0, 0, cos[i], sen[i]]])
  Be.append(B)
for i, matriz in enumerate(Be):
    print(f"B{i + 1}:\n{matriz}\n")

B1:
[[0.8 0.6 0.  0. ]
 [0.  0.  0.8 0.6]]

B2:
[[0.6 0.8 0.  0. ]
 [0.  0.  0.6 0.8]]

B3:
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]]



In [ ]:
u = []
ue = []
for i in range(len(Be)):
    matriz_Be = Be[i]
    vetor_desl_extre = deslocamentos_extremidade[i]
    u = np.dot(matriz_Be, vetor_desl_extre)
    ue.append(u)
for i, valores in enumerate(ue):
    print(f"u{i+1}: {valores}")

u1: [0.   0.39]
u2: [0.   0.38]
u3: [0.   0.25]


In [ ]:
delta_Le = []

for i, valores in enumerate(ue):
    diferenca = valores[-1] - valores[0]
    delta_Le.append(diferenca)
    if delta_Le[i] > 0:
      print(f"ΔL{i+1}: {diferenca:.2f} alongamento")
    else:
      print(f"ΔL{i+1}: {diferenca:.2f} encurtamento")

ΔL1: 0.39 alongamento
ΔL2: 0.38 alongamento
ΔL3: 0.25 alongamento


14. Esforços normais

In [ ]:
Ee_Ae_Be_De = []
N = []
Ne = []
for i in range (elementos_count):
  N = E[i]*A[i]/L[i] * np.array([[1,-1],[-1,1]])
  Ee_Ae_Be_De_i = E[i]*A[i]*beta[i]*(delta[i] if i < len(delta) else 0) * np.array([-1,1])
  Ee_Ae_Be_De.append(Ee_Ae_Be_De_i)
  Ne.append(np.dot(N, ue[i]) - Ee_Ae_Be_De[i])

for i, esforco_normal in enumerate(Ne):
    print(f"N{i + 1}:\n{esforco_normal}\n")

ultimos_valores = [vetor[-1] for vetor in Ne]
for i, valor in enumerate(ultimos_valores):
    if valor > 0:
        print(f"N{i + 1}: {valor:.2f} Tração")
    elif valor < 0:
        print(f"N{i + 1}: {valor:.2f} Compressão")
    else:
        print(f"N{i + 1}: {valor:.2f} Equilíbrio")

N1:
[ 7.5 -7.5]

N2:
[-80.  80.]

N3:
[-12.5  12.5]

N1: -7.50 Compressão
N2: 80.00 Tração
N3: 12.50 Tração


15. Reações de Apoio

In [ ]:
Nr = []
for i in range(elementos_count):
    Nr.append([Ne[i][0] * cos[i], Ne[i][0] * sen[i], Ne[i][1] * cos[i], Ne[i][1] * sen[i],])

for i, reacoes_apoio in enumerate(Nr):
    print(f"N{i + 1}:\n{reacoes_apoio}\n")

N1:
[6.000000000000046, 4.500000000000034, -6.000000000000046, -4.500000000000034]

N2:
[-48.000000000000036, -64.00000000000004, 48.000000000000036, 64.00000000000004]

N3:
[-0.0, -12.500000000000057, 0.0, 12.500000000000057]



In [ ]:
novo_dicionario = {}

for i, posicoes in enumerate(e):
    vetor = [0] * (2*nos)
    for j, pos in enumerate(posicoes):
        vetor[pos - 1] = Nr[i][j]
    novo_dicionario[f"N{i + 1}"] = vetor
vetor_soma = [0] * (2*nos)
for vetor in novo_dicionario.values():
    vetor_soma = [v1 + v2 for v1, v2 in zip(vetor_soma, vetor)]

for chave, vetor in novo_dicionario.items():
    print(f"{chave}: {vetor}")

FR = []
for i in range (2*nos):
  FR.append(-Fc[i]+vetor_soma[i])
FR_formatado = [f"{valor:.3f}" for valor in FR]
print("FR: ", FR_formatado)

N1: [6.000000000000046, 4.500000000000034, 0, 0, 0, 0, -6.000000000000046, -4.500000000000034]
N2: [0, 0, -48.000000000000036, -64.00000000000004, 0, 0, 48.000000000000036, 64.00000000000004]
N3: [0, 0, 0, 0, -0.0, -12.500000000000057, 0.0, 12.500000000000057]
FR:  ['6.000', '4.500', '-48.000', '-64.000', '0.000', '-12.500', '-0.000', '0.000']
